In [1]:
from secrets import OMDB_API_KEY, HADOOP_USER_NAME, SPARK_URI, HADOOP_NAMENODE

In [2]:
import os
os.environ['HADOOP_USER_NAME'] = HADOOP_USER_NAME

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql.functions as F
from hdfs import InsecureClient
import omdb
from omdb import OMDBClient
import pyspark.sql.types as t

In [4]:
client_hdfs = InsecureClient(f'http://{HADOOP_NAMENODE}:50070', user=HADOOP_USER_NAME)

In [5]:
# get preprocessed opusdata filename
hdfs_path = "/processed/opusdata.csv"

filename = [f for f in client_hdfs.list(hdfs_path) if f.endswith('.csv')][0]

In [6]:
sc = SparkContext(SPARK_URI)
sparkSession = (
    SparkSession.builder.appName("preprocessing-opusdata-and-omdb")
    .config("spark.hadoop.dfs.client.use.datanode.hostname", "true")
    .getOrCreate()
)

['/home/utente/spark-2.4.5-bin-hadoop2.7/./bin/spark-submit', 'pyspark-shell'] {'CONDA_SHLVL': '2', 'LS_COLORS': 'rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=01;31:*.tzo=01;31:*.t7z=01;31:*.zip=01;31:*.z=01;31:*.Z=01;31:*.dz=01;31:*.gz=01;31:*.lrz=01;31:*.lz=01;31:*.lzo=01;31:*.xz=01;31:*.zst=01;31:*.tzst=01;31:*.bz2=01;31:*.bz=01;31:*.tbz=01;31:*.tbz2=01;31:*.tz=01;31:*.deb=01;31:*.rpm=01;31:*.jar=01;31:*.war=01;31:*.ear=01;31:*.sar=01;31:*.rar=01;31:*.alz=01;31:*.ace=01;31:*.zoo=01;31:*.cpio=01;31:*.7z=01;31:*.rz=01;31:*.cab=01;31:*.wim=01;31:*.swm=01;31:*.dwm=01;31:*.esd=01;31:*.jpg=01;35:*.jpeg=01;35:*.mjpg=01;35:*.mjpeg=01;35:*.gif=01;35:*.bmp=01;35:*.pbm=01;35:*.pgm=01;35:*.ppm=01;35:*.tga=01;35:*.xbm=01;35:*.xpm=01;35:*.tif=01;35:*.tiff=01

In [7]:
# Read from hdfs
opusdata = sparkSession.read.csv(
    f"hdfs://{HADOOP_NAMENODE}:8020{hdfs_path}/{filename}", header=True, inferSchema=True
)


In [8]:
opusdata.show()

+--------------------+---------------+-----------------+------+-----------------+------+----------------+-------+
|          movie_name|production_year|production_budget|rating|            genre|sequel|total_box_office|success|
+--------------------+---------------+-----------------+------+-----------------+------+----------------+-------+
|       Runner Runner|           2013|         30000000|     R|Thriller/Suspense|     0|        60512680|      1|
|The Hunger Games:...|           2015|        160000000| PG-13|Thriller/Suspense|     1|       648986787|      1|
|   Chasing Mavericks|           2012|         20000000|    PG|            Drama|     0|         8300821|      0|
|        Darkest Hour|           2017|         30000000| PG-13|            Drama|     0|       150355828|      1|
|       Inherent Vice|           2014|         20000000|     R|            Drama|     0|        14772346|      0|
|       Live by Night|           2015|         65000000|     R|            Drama|     0|

In [9]:
opusdata.count()

1163

In [10]:
omdb.set_default('apikey', OMDB_API_KEY)
client = OMDBClient(apikey=OMDB_API_KEY)

In [11]:
requested_flat_fields = ['runtime', 'director', 'actors', 'country', 'awards', 'imdb_votes', 'imdb_id']
requested_nested_fields = {'ratings': ['Internet Movie Database', 'Rotten Tomatoes', 'Metacritic']}

In [12]:
def format_source(source):
    return '_'.join(source.split()).lower()

In [13]:
def construct_schema(requested_flat_fields, requested_nested_fields):
    schema = []
    for key in requested_flat_fields:
        schema.append(t.StructField(key, t.StringType(), True))
    for key, values in requested_nested_fields.items():
        for value in values:
            schema.append(t.StructField(f'{key}_{format_source(value)}', t.StringType(), True))
            
    return t.StructType(schema)

In [14]:
schema = construct_schema(requested_flat_fields, requested_nested_fields)

In [15]:
@F.udf(returnType=schema)
def omdb_data(arguments):

    movie_name, year = arguments
    print(movie_name)
    result = client.get(title=movie_name, year=year, fullplot=True, tomatoes=True)
    
    result_to_keep = {}
    
    for key in requested_flat_fields:
        result_to_keep[key] = result.get(key, None)
        
    for nested_field in requested_nested_fields:
        requested_nested_list = requested_nested_fields[nested_field]
        nested_list = result.get(nested_field, None)
        
        if nested_list:
            for nested_dict in nested_list:
                source = nested_dict.get('source', None)

                if source:
                    value = nested_dict.get('value', None)
                    
                    if source in requested_nested_list:

                        source_formatted = format_source(source)
                        key = f'{nested_field}_{source_formatted}'

                        result_to_keep[key] = value
                        
            requested_sources = requested_nested_fields[nested_field]
            for requested_source in requested_sources:
                source_formatted = format_source(requested_source)
                key = f'{nested_field}_{source_formatted}'
                if not key in result_to_keep:
                    result_to_keep[key] = None
                    
        else:
            requested_sources = requested_nested_fields[nested_field]
            for requested_source in requested_sources:
                source_formatted = format_source(requested_source)
                key = f'{nested_field}_{source_formatted}'
                result_to_keep[key] = None
                
            


    #print(result_to_keep.keys(), result_to_keep.values())
    return t.Row(*list(result_to_keep.keys()))(*list(result_to_keep.values()))
    

In [16]:
opusdata_omdb = opusdata.withColumn(
    "omdb_data", F.explode(F.array(omdb_data(F.array("movie_name", "production_year"))))
)

In [17]:
opusdata_fields_name = [field.name for field in opusdata.schema.fields]

In [18]:
opusdata_ombd = opusdata_omdb.select(*opusdata_fields_name, 'omdb_data.*')

In [19]:
opusdata_ombd_id_not_null = opusdata_ombd.na.drop(
    subset=[
        "imdb_id",
        "ratings_internet_movie_database",
        "ratings_rotten_tomatoes",
        "ratings_metacritic",
    ]
)

In [20]:
opusdata_ombd_no_id_duplicated = opusdata_ombd_id_not_null.dropDuplicates(['imdb_id'])

### Processing awards

In [21]:
@F.udf(returnType=t.IntegerType())
def general_awards_by_keyword(awards_str, keyword):
    n_nominations = awards_str.split(keyword)[0].split()[-1]
    try:
        n_nominations_int = int(n_nominations)
    except ValueError as e:
        n_nominations_int = 0
    return n_nominations_int

In [22]:
keywords_general = ['nomination', 'win']

In [23]:
awards_name = ['golden globe', 'oscar', 'bafta']

In [24]:
@F.udf(returnType=t.IntegerType())
def won_by_keyword(awards_str, award_name):
    awards_str = awards_str.lower()
    
    try:
        won_or_nominated = awards_str.split(award_name)[0].split()[-2]
        if won_or_nominated == "won":
            n_won = int(awards_str.split(award_name)[0].split()[-1])
        else:
            n_won = 0
    except IndexError as e:
        n_won = 0

    return n_won

In [25]:
@F.udf(returnType=t.IntegerType())
def nominated_by_keyword(awards_str, award_name):
    awards_str = awards_str.lower()
   
    try:
        won_or_nominated = awards_str.split(award_name)[0].split()[-2]
        if won_or_nominated == "for":
            n_nominated = int(awards_str.split(award_name)[0].split()[-1])
        else:
            n_nominated = 0
    except IndexError as e:
        n_nominated = 0

    return n_nominated

In [26]:
opusdata_awards_categorized = opusdata_ombd_no_id_duplicated
for general_keyword in keywords_general:
    opusdata_awards_categorized = opusdata_awards_categorized.withColumn(
        f'{general_keyword}s', general_awards_by_keyword("awards", F.lit(general_keyword))
    )

In [27]:
for award_name in awards_name:
    award_name_formatted = '_'.join(award_name.split())
    opusdata_awards_categorized = opusdata_awards_categorized.withColumn(
        f'won_{award_name_formatted}s', won_by_keyword("awards", F.lit(award_name))
    )
    opusdata_awards_categorized = opusdata_awards_categorized.withColumn(
        f'nominated_{award_name_formatted}s', nominated_by_keyword("awards", F.lit(award_name))
    )

In [28]:
opusdata_awards_categorized = opusdata_awards_categorized.drop('awards')

In [29]:
opusdata_awards_categorized.show()

Py4JJavaError: An error occurred while calling o170.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 4 times, most recent failure: Lost task 0.3 in stage 5.0 (TID 8, 10.0.0.8, executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/utente/spark-2.4.5-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/utente/spark-2.4.5-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/utente/spark-2.4.5-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 352, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/utente/spark-2.4.5-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 142, in dump_stream
    for obj in iterator:
  File "/home/utente/spark-2.4.5-bin-hadoop2

### Scale rankings [0..1]

In [ ]:
# scale imdb ratings
opusdata_scaled_ratings = opusdata_awards_categorized.withColumn(
    "ratings_internet_movie_database", F.split(F.col("ratings_internet_movie_database"), "/").cast("array<float>") \
   
)
opusdata_scaled_ratings = opusdata_scaled_ratings.withColumn(
    "ratings_internet_movie_database", F.col("ratings_internet_movie_database")[0] / 10
)

In [ ]:
# scale rotten tomatoes ratings
opusdata_scaled_ratings = opusdata_scaled_ratings.withColumn(
    "ratings_rotten_tomatoes", F.split(F.col("ratings_rotten_tomatoes"), "%").cast("array<int>") \
   
)
opusdata_scaled_ratings = opusdata_scaled_ratings.withColumn(
    "ratings_rotten_tomatoes", F.col("ratings_rotten_tomatoes")[0] / 100
)

In [ ]:
# scale metacritic ratings
opusdata_scaled_ratings = opusdata_scaled_ratings.withColumn(
    "ratings_metacritic", F.split(F.col("ratings_metacritic"), "/").cast("array<int>") \
   
)
opusdata_scaled_ratings = opusdata_scaled_ratings.withColumn(
    "ratings_metacritic", F.col("ratings_metacritic")[0] / 100
)

In [ ]:
# remove comma from imdb_votes
opusdata_votes = opusdata_scaled_ratings.withColumn(
    "imdb_votes", F.regexp_replace("imdb_votes", ",", "")
)

### Encode actors

In [ ]:
unique_actors = set()

for i, row in enumerate(opusdata_votes.rdd.collect()):
    actors = row['actors']
    unique_actors.update([a.strip().lower() for a in actors.split(',')])    

In [ ]:
actors_id_dict = {actor: i for i, actor in enumerate(unique_actors)}

In [ ]:
schema_actors = t.StructType([
    t.StructField('actor_id_0', t.IntegerType(), True),
    t.StructField('actor_id_1', t.IntegerType(), True),
    t.StructField('actor_id_2', t.IntegerType(), True),
    t.StructField('actor_id_3', t.IntegerType(), True)
])

In [ ]:
@F.udf(returnType=schema_actors)
def encode_authors(actors_str):
    actors = [a.strip().lower() for a in actors_str.split(',')]
    
    ids = []
    for a in actors:
        ids.append(actors_id_dict[a])
    
    ids = sorted(ids) + (4-len(ids))*[None]
        
    return t.Row('actor_id_0', 'actor_id_1', 'actor_id_2', 'actor_id_3')(*ids)

In [ ]:
opusdata_actors = opusdata_votes.withColumn(
    "actors_ids", F.explode(F.array(encode_authors("actors")))
)

opusdata_fields_name = [
    field.name
    for field in opusdata_actors.schema.fields
    if field.name != "actors_ids" and field.name != "actors"
]
opusdata_actors = opusdata_actors.select(*opusdata_fields_name, "actors_ids.*")

### Runtime - remove "min"

In [ ]:
opusdata_runtime = opusdata_actors.withColumn(
    "runtime", F.split(F.col("runtime"), " ").cast("array<string>")
)
opusdata_runtime = opusdata_runtime.withColumn("runtime", F.col("runtime")[0])

### Keep only first country

In [ ]:
opusdata_first_country = opusdata_runtime.withColumn(
    "country", F.split(F.col("country"), ",").cast("array<string>") \
   
)
opusdata_first_country = opusdata_first_country.withColumn(
    "country", F.col("country")[0]
)

### Keep only first director

In [ ]:
opusdata_first_director = opusdata_first_country.withColumn(
    "director", F.split(F.col("director"), ",").cast("array<string>") \
   
)
opusdata_first_director = opusdata_first_director.withColumn(
    "director", F.col("director")[0]
)

In [ ]:
opusdata_first_director.repartition(1).write.mode("overwrite").option('header',True).csv(
    f"hdfs://{HADOOP_NAMENODE}:8020/processed/opusdata_omdb.csv"
)